In [5]:
# Import the simulation and some packages we need
import simulation
import numpy as np
import pandas as pd

In [8]:
np.random.seed(37) # Set a random seed for reproducibility

# Generate a demand for two simulations, assume Poisson(5) and time horizon of T=10
demand_1 = np.random.poisson(5, 10)
demand_2 = np.random.poisson(5, 10)

In [9]:
# Initialise the simulation with thr following parameters
# T=10
# L=2
# c_u=9
# c_o=1
# c=0
# init_inv = [5,5,5]
# Order_rule = FQ
sim = simulation.Simulation(10,2,9,1,0,[5,5,5], 'FQ')

In [10]:
# Run the first simulation
sim.run(demand_1)
display(sim.log)
print('Simulation 1, total cost: {}'.format(np.sum(sim.period_cost)))

,StartingInv,Order,PostOrder,Demand,PostDemand,PostDeliveryMovements,PeriodCost
Period,,,,,,,
1,"[5.0, 5.0, 5.0]",5,"[5.0, 5.0, 5.0]",6,-1.0,"[5.0, 5.0, 0.0]",9.0
2,"[5.0, 5.0, 0.0]",5,"[5.0, 5.0, 5.0]",8,-3.0,"[5.0, 5.0, 0.0]",27.0
3,"[5.0, 5.0, 0.0]",5,"[5.0, 5.0, 5.0]",5,0.0,"[5.0, 5.0, 0.0]",0.0
4,"[5.0, 5.0, 0.0]",5,"[5.0, 5.0, 5.0]",5,0.0,"[5.0, 5.0, 0.0]",0.0
5,"[5.0, 5.0, 0.0]",5,"[5.0, 5.0, 5.0]",5,0.0,"[5.0, 5.0, 0.0]",0.0
6,"[5.0, 5.0, 0.0]",5,"[5.0, 5.0, 5.0]",3,2.0,"[7.0, 5.0, 0.0]",2.0
7,"[7.0, 5.0, 0.0]",5,"[7.0, 5.0, 5.0]",2,5.0,"[10.0, 5.0, 0.0]",5.0
8,"[10.0, 5.0, 0.0]",5,"[10.0, 5.0, 5.0]",8,2.0,"[7.0, 5.0, 0.0]",2.0
9,"[7.0, 5.0, 0.0]",5,"[7.0, 5.0, 5.0]",10,-3.0,"[5.0, 5.0, 0.0]",27.0


Simulation 1, total cost: 105.0


In [11]:
# Reset the simulation so we can rerun it
sim.reset()

In [12]:
sim.run(demand_2)
display(sim.log)
print('Simulation 2, total cost: {}'.format(np.sum(sim.period_cost)))

,StartingInv,Order,PostOrder,Demand,PostDemand,PostDeliveryMovements,PeriodCost
Period,,,,,,,
1,"[5.0, 5.0, 5.0]",5,"[5.0, 5.0, 5.0]",9,-4.0,"[5.0, 5.0, 0.0]",36.0
2,"[5.0, 5.0, 0.0]",5,"[5.0, 5.0, 5.0]",5,0.0,"[5.0, 5.0, 0.0]",0.0
3,"[5.0, 5.0, 0.0]",5,"[5.0, 5.0, 5.0]",4,1.0,"[6.0, 5.0, 0.0]",1.0
4,"[6.0, 5.0, 0.0]",5,"[6.0, 5.0, 5.0]",4,2.0,"[7.0, 5.0, 0.0]",2.0
5,"[7.0, 5.0, 0.0]",5,"[7.0, 5.0, 5.0]",6,1.0,"[6.0, 5.0, 0.0]",1.0
6,"[6.0, 5.0, 0.0]",5,"[6.0, 5.0, 5.0]",8,-2.0,"[5.0, 5.0, 0.0]",18.0
7,"[5.0, 5.0, 0.0]",5,"[5.0, 5.0, 5.0]",4,1.0,"[6.0, 5.0, 0.0]",1.0
8,"[6.0, 5.0, 0.0]",5,"[6.0, 5.0, 5.0]",3,3.0,"[8.0, 5.0, 0.0]",3.0
9,"[8.0, 5.0, 0.0]",5,"[8.0, 5.0, 5.0]",9,-1.0,"[5.0, 5.0, 0.0]",9.0


Simulation 2, total cost: 89.0
